# Импорт библиотек и выгрузка данных

In [6]:
import os
import json
import warnings
import re
import codecs
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from multiprocessing import Pool
from functools import reduce
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.base import TransformerMixin
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set_style('whitegrid')
sns.set_palette('Set2')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\simbo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
train = pd.read_csv('data/cutedwdntrain.csv',sep = ';', index_col='id')
train = shuffle(train, random_state=42)

In [8]:
train

,text,rating,length,label,len
id,,,,,
16334,почти месяц наушник пока хорошо работать микро...,4,50-100,0.75,67
38246,писать огромный комментарий о тот какой они кр...,5,250-500,1.00,402
30901,при чем тут фирма вообще грязный хлипкий наушн...,1,50-100,0.00,62
71437,полный отстой правый наушник не проработать и ...,1,0-50,0.00,50
46508,прислать быстро но один наушник не работать,3,0-50,0.50,43
...,...,...,...,...,...
42881,прислать в срок упаковка на звук хороший есть ...,3,100-150,0.50,133
56629,наушник неплохой прислать хорошо упаковать вну...,4,100-150,0.75,115
41498,хороший наушник интересный дизайн и хороший звук,5,0-50,1.00,48


## Разбиение данных и доп функции

In [9]:
X = train['text']
y = train['label']

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.8, random_state=42, stratify= y)

In [12]:
test_y.value_counts()

1.00    965
0.00    964
0.75    715
0.50    480
0.25    365
Name: label, dtype: int64

In [13]:
train_X

id
7778     наушник прислать в срок но хватать зарядка мин...
32506    хороший наушник звук классный пока не понимать...
19958    прийти браковать товар шуметь наушник без музы...
2678     наушник прислать в срок но хватать зарядка мин...
33709    правый наушник не работать проверять сразу на ...
                               ...                        
18303    сегодня забрать товар всё подключиться звук хо...
69625    качество звук ужасный очень глухо когда открыт...
64394             звук качественный зарядка хватать на час
28080    наушник классный вс работать но как палра банк...
19749    брать уже в третий раз хватать на три месяц по...
Name: text, Length: 13956, dtype: object

In [14]:
stop_words = stopwords.words('russian')
stop_words

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [15]:
def make_pipeline(vectorizer, transformer, classifier):
    return Pipeline([
            ('vectorizer', vectorizer),
            ('transformer', transformer),
            ('classifier', classifier)
        ])

In [16]:
def make_estimator(classifier, params_grid, scorer, data, labels):
    pipeline = make_pipeline(CountVectorizer(), TfidfTransformer(), classifier)
    grid_cv = RandomizedSearchCV(pipeline, params_grid, scoring=scorer, cv=5, random_state=777, n_iter=100)
    grid_cv.fit(data, labels)
    return grid_cv

In [17]:
class DenseTransformer(TransformerMixin):
    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

# Моделирование по двум меткам 1 и 5

In [18]:
Xy = pd.merge(train_X,train_y,on = 'id')

In [19]:
Xytest = pd.merge(test_X,test_y,on = 'id')

In [20]:
train15 =  shuffle(pd.concat([Xy[Xy['label']==0],Xy[Xy['label']==1]]),random_state=42)
train15['label'] = train15['label'].apply(lambda x: int(x))

In [21]:
test15 =  shuffle(pd.concat([Xytest[Xytest['label']==0],Xytest[Xytest['label']==1]]),random_state=42)
test15['label'] = test15['label'].apply(lambda x: int(x))

In [22]:
test15.describe()

,label
count,1929.000000
mean,0.500259
std,0.500130
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [23]:
X15 = train15['text'].values
y15 = train15['label'].values

In [24]:
X15test = test15['text'].values
y15test = test15['label'].values

# Линейные модели

In [25]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier

In [22]:
%%time
for name, clf in {'LogisticRegression': LogisticRegression, 'LinearSVC': LinearSVC,
               'SGDClassifier': SGDClassifier, 'RidgeClassifier': RidgeClassifier}.items():
    score = cross_val_score(make_pipeline(CountVectorizer(), TfidfTransformer(), clf(random_state=777)), X15, y15, cv=5).mean()
    print(f"{name} - {score}")

LogisticRegression - 0.9551348166229985
LinearSVC - 0.9665457910836185
SGDClassifier - 0.9651192406525265
RidgeClassifier - 0.9678430601192112
CPU times: total: 3.75 s
Wall time: 5.51 s


In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier

## Логистическая регрессия

In [42]:
vectorizer = CountVectorizer(min_df=10)
XX = vectorizer.fit_transform(X15)

In [43]:
transformer = TfidfTransformer()
Xlr = transformer.fit_transform(XX)

In [44]:
%%time
log_reg = LogisticRegression()
log_reg.fit(Xlr, y15)

CPU times: total: 78.1 ms
Wall time: 87 ms


LogisticRegression()

In [46]:
coeffs = log_reg.coef_[0]
worsestwords = [vectorizer.get_feature_names()[list(coeffs).index(i)] for i in sorted(coeffs,reverse=True)[:30]]
worsestwords

['отличный',
 'хороший',
 'спасибо',
 'довольный',
 'всё',
 'отлично',
 'хорошо',
 'супер',
 'понравиться',
 'классный',
 'чистый',
 'нравиться',
 'цена',
 'восторг',
 'целый',
 'быстро',
 'рекомендовать',
 'удобный',
 'свой',
 'пока',
 'проблема',
 'пожалеть',
 'качественный',
 'удобно',
 'топ',
 'огонь',
 'немного',
 'для',
 'крутой',
 'прекрасный']

In [26]:
log_reg.score(transformer.transform(vectorizer.transform(X15test)),y15test)

0.9595645412130638

## Настройка параметров линейных моделей

In [30]:
params_grid_vectorizer = {
    'vectorizer__max_df' : [0.85, 0.9, 0.95, 1.0],
    'vectorizer__min_df' : [1, 10, 20],
    'vectorizer__ngram_range' : [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6)],
    'vectorizer__stop_words' : [stop_words, None, 'russian']
}

In [31]:
params_grid_lr = {
    'classifier__C': np.arange(0.1, 2, 0.1),
    'classifier__max_iter': np.arange(50, 500, 50),
    'classifier__solver': ['lbfgs', 'liblinear', 'sag']
}
params_grid_lsvc = {
    'classifier__loss': ['hinge', 'squared_hinge'],
    'classifier__max_iter': np.arange(100, 1000, 50),
    'classifier__tol': [1e-5, 1e-4, 1e-3],
    'classifier__C': np.arange(0.1, 2, 0.1)
}
params_grid_sgdc = {
    'classifier__loss': ['log', 'hinge', 'modified_huber'],
    'classifier__penalty':  ['l1', 'l2', 'elasticnet'],
    'classifier__max_iter': np.arange(100, 1000, 50),
    'classifier__tol': np.arange(1e-5, 1e-3, 1e-5)
}
params_grid_rc = {
    'classifier__alpha': np.arange(0.1, 5, 0.1),
    'classifier__normalize': [True, False],
    'classifier__tol': np.arange(1e-4, 1e-2, 1e-4),
    'classifier__solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag']
}

In [32]:
%%time
grid_search_lr = make_estimator(LogisticRegression(random_state = 42),
                                {**params_grid_vectorizer, **params_grid_lr}, 'accuracy', X15, y15)
print("LogisticRegression:")
print(f"Лучшее качество - {grid_search_lr.best_score_}")
print(f"Параметры - {grid_search_lr.best_params_}")

LogisticRegression:
Лучшее качество - 0.9684914004335718
Параметры - {'vectorizer__stop_words': None, 'vectorizer__ngram_range': (1, 2), 'vectorizer__min_df': 1, 'vectorizer__max_df': 1.0, 'classifier__solver': 'liblinear', 'classifier__max_iter': 50, 'classifier__C': 1.9000000000000001}
CPU times: total: 5min 55s
Wall time: 6min 22s


In [33]:

%%time
grid_search_lsvc = make_estimator(LinearSVC(random_state=777),
                                  {**params_grid_vectorizer, **params_grid_lsvc}, 'accuracy', X15, y15)
print("LinearSVC:")
print(f"Лучшее качество - {grid_search_lsvc.best_score_}")
print(f"Параметры - {grid_search_lsvc.best_params_}")

LinearSVC:
Лучшее качество - 0.9748451018910556
Параметры - {'vectorizer__stop_words': None, 'vectorizer__ngram_range': (1, 2), 'vectorizer__min_df': 1, 'vectorizer__max_df': 0.95, 'classifier__tol': 0.0001, 'classifier__max_iter': 550, 'classifier__loss': 'squared_hinge', 'classifier__C': 1.2000000000000002}
CPU times: total: 3min 32s
Wall time: 3min 36s


In [31]:
%%time
grid_search_sgdc = make_estimator(SGDClassifier(random_state=777),
                                  {**params_grid_vectorizer, **params_grid_sgdc}, 'accuracy', X15, y15)
print("SGDClassifier:")
print(f"Лучшее качество - {grid_search_sgdc.best_score_}")
print(f"Параметры - {grid_search_sgdc.best_params_}")

SGDClassifier:
Лучшее качество - 0.974196425344197
Параметры - {'vectorizer__stop_words': None, 'vectorizer__ngram_range': (1, 2), 'vectorizer__min_df': 1, 'vectorizer__max_df': 0.85, 'classifier__tol': 0.0003400000000000001, 'classifier__penalty': 'l2', 'classifier__max_iter': 200, 'classifier__loss': 'hinge'}
CPU times: total: 3min 38s
Wall time: 3min 48s


In [32]:
%%time
grid_search_rc = make_estimator(RidgeClassifier(random_state=777),
                                {**params_grid_vectorizer, **params_grid_rc}, 'accuracy', X15, y15)
print("RidgeClassifier:")
print(f"Лучшее качество - {grid_search_rc.best_score_}")
print(f"Параметры - {grid_search_rc.best_params_}")

RidgeClassifier:
Лучшее качество - 0.9721222070637404
Параметры - {'vectorizer__stop_words': None, 'vectorizer__ngram_range': (1, 4), 'vectorizer__min_df': 1, 'vectorizer__max_df': 0.95, 'classifier__tol': 0.0015, 'classifier__solver': 'lsqr', 'classifier__normalize': False, 'classifier__alpha': 0.4}
CPU times: total: 3min 33s
Wall time: 3min 42s


In [33]:
results_linear = {'LogisticRegression': (grid_search_lr.best_score_, grid_search_lr.best_params_),
                  'LinearSVC': (grid_search_lsvc.best_score_, grid_search_lsvc.best_params_),
                  'SGDClassifier': (grid_search_sgdc.best_score_, grid_search_sgdc.best_params_),
                  'RidgeClassifier': (grid_search_rc.best_score_, grid_search_rc.best_params_)}
print(f"Лучшая линейная модель - {max(results_linear, key=results_linear.get)}")
print(f"Лучшее качество - {max(results_linear.values())[0]}")
print(f"Параметры - {max(results_linear.values())[1]}")

Лучшая линейная модель - LinearSVC
Лучшее качество - 0.9748451018910556
Параметры - {'vectorizer__stop_words': None, 'vectorizer__ngram_range': (1, 2), 'vectorizer__min_df': 1, 'vectorizer__max_df': 0.95, 'classifier__tol': 0.0001, 'classifier__max_iter': 550, 'classifier__loss': 'squared_hinge', 'classifier__C': 1.2000000000000002}


# Решающие деревья

In [34]:
%%time
for name, clf in {'DecisionTreeClassifier': DecisionTreeClassifier, 'RandomForestClassifier': RandomForestClassifier,
               'BaggingClassifier': BaggingClassifier, 'GradientBoostingClassifier': GradientBoostingClassifier}.items():
    score = cross_val_score(make_pipeline(CountVectorizer(), TfidfTransformer(), clf(random_state=777)), X15, y15, cv=5).mean()
    print(f"{name} - {score}")

DecisionTreeClassifier - 0.9109183938509802
RandomForestClassifier - 0.9614888543129803
BaggingClassifier - 0.9343886830865807
GradientBoostingClassifier - 0.9238860407194366
CPU times: total: 1min 34s
Wall time: 2min 3s


In [36]:
params_grid_dtc = {
    'classifier__max_depth': np.arange(1, 30, 1),
    'classifier__min_samples_split': np.arange(2, 6, 1),
    'classifier__min_samples_leaf': [1, 2, 3],
    'classifier__min_impurity_decrease': np.logspace(-10, -6, 5)
}
params_grid_rfc = {
    'classifier__n_estimators': np.arange(1, 30, 1),
    'classifier__max_depth': [None] + list(np.arange(1, 30, 1)),
    'classifier__min_samples_split': np.arange(2, 6, 1),
    'classifier__min_samples_leaf': [1, 2, 3],
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__min_impurity_decrease': np.logspace(-10, -6, 5)
}
params_grid_bc = {
    'classifier__n_estimators': np.arange(10, 1000, 10),
    'classifier__warm_start': [False, True],
    'classifier__bootstrap_features': [False, True]
}
params_grid_gbc = {
    'classifier__n_estimators': np.arange(10, 1000, 10),
    'classifier__max_depth': [None] + list(np.arange(1, 30, 1)),
    'classifier__min_samples_split': np.arange(2, 6, 1),
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__subsample': np.arange(0.6, 1, 0.1)
}

In [41]:
%%time
grid_search_dtc = make_estimator(DecisionTreeClassifier(random_state=777),
                                {**params_grid_vectorizer, **params_grid_dtc}, 'accuracy', X15, y15)
print("DecisionTreeClassifier:")
print(f"Лучшее качество - {grid_search_dtc.best_score_}")
print(f"Параметры - {grid_search_dtc.best_params_}")

DecisionTreeClassifier:
Лучшее качество - 0.9068973053487024
Параметры - {'vectorizer__stop_words': None, 'vectorizer__ngram_range': (1, 3), 'vectorizer__min_df': 20, 'vectorizer__max_df': 1.0, 'classifier__min_samples_split': 3, 'classifier__min_samples_leaf': 3, 'classifier__min_impurity_decrease': 1e-08, 'classifier__max_depth': 29}
CPU times: total: 10min 1s
Wall time: 11min


In [47]:
%%time
grid_search_rfc = make_estimator(RandomForestClassifier(random_state=777),
                                {**params_grid_vectorizer, **params_grid_rfc}, 'accuracy', X15, y15)
print("RandomForestClassifier:")
print(f"Лучшее качество - {grid_search_rfc.best_score_}")
print(f"Параметры - {grid_search_rfc.best_params_}")

RandomForestClassifier:
Лучшее качество - 0.948392850688394
Параметры - {'vectorizer__stop_words': None, 'vectorizer__ngram_range': (1, 2), 'vectorizer__min_df': 10, 'vectorizer__max_df': 1.0, 'classifier__n_estimators': 28, 'classifier__min_samples_split': 3, 'classifier__min_samples_leaf': 1, 'classifier__min_impurity_decrease': 1e-08, 'classifier__max_features': 'auto', 'classifier__max_depth': 28}
CPU times: total: 5min 25s
Wall time: 5min 37s


In [22]:
results_tree = {'DecisionTreeClassifier': (grid_search_dtc.best_score_, grid_search_dtc.best_params_),
                'RandomForestClassifier': (grid_search_rfc.best_score_, grid_search_rfc.best_params_)}
print(f"Лучшая модель решающих деревьев - {max(results_tree, key=results_tree.get)}")
print(f"Лучшее качество - {max(results_tree.values())[0]}")
print(f"Параметры - {max(results_tree.values())[1]}")

NameError: name 'grid_search_dtc' is not defined

## Наивный Байес


In [23]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB

In [24]:
%%time
for name, clf in {'MultinomialNB': MultinomialNB, 'BernoulliNB': BernoulliNB,
               'GaussianNB': GaussianNB}.items():
    if name == 'GaussianNB':
        score = cross_val_score(Pipeline([('vectorizer', CountVectorizer()), ('transformer', TfidfTransformer()),
                                  ('dense', DenseTransformer()), ('classifier', GaussianNB())]), X15, y15, cv=5).mean()
    else:
        score = cross_val_score(make_pipeline(CountVectorizer(), TfidfTransformer(), clf()), X15, y15, cv=5).mean()
    print(f"{name} - {score}")

MultinomialNB - 0.9509855394808403
BernoulliNB - 0.9561736069257171
GaussianNB - 0.8603414609133925
CPU times: total: 10.4 s
Wall time: 13.6 s


In [28]:
params_grid_mnb = {
    'classifier__alpha': np.logspace(0, 5, 100),
    'classifier__fit_prior': [True, False]
}
params_grid_bnb = {
    'classifier__alpha': np.logspace(0, 5, 100),
    'classifier__fit_prior': [True, False]
}


In [27]:
%%time
grid_search_mnb = make_estimator(MultinomialNB(),
                                {**params_grid_vectorizer, **params_grid_mnb}, 'accuracy', X15, y15)
print("MultinomialNB:")
print(f"Лучшее качество - {grid_search_mnb.best_score_}")
print(f"Параметры - {grid_search_mnb.best_params_}")


NameError: name 'params_grid_vectorizer' is not defined

In [51]:
%%time
grid_search_bnb = make_estimator(BernoulliNB(),
                                {**params_grid_vectorizer, **params_grid_bnb}, 'accuracy', X15, y15)
print("BernoulliNB:")
print(f"Лучшее качество - {grid_search_bnb.best_score_}")
print(f"Параметры - {grid_search_bnb.best_params_}")

BernoulliNB:
Лучшее качество - 0.9617490982664693
Параметры - {'vectorizer__stop_words': None, 'vectorizer__ngram_range': (1, 2), 'vectorizer__min_df': 10, 'vectorizer__max_df': 0.9, 'classifier__fit_prior': False, 'classifier__alpha': 1.0}
CPU times: total: 3min 59s
Wall time: 4min 4s


In [52]:
results_bayes = {'MultinomialNB': (grid_search_mnb.best_score_, grid_search_mnb.best_params_),
                 'BernoulliNB': (grid_search_bnb.best_score_, grid_search_bnb.best_params_)}
print(f"Лучшая байесовская модель - {max(results_bayes, key=results_bayes.get)}")
print(f"Лучшее качество - {max(results_bayes.values())[0]}")
print(f"Параметры - {max(results_bayes.values())[1]}")

Лучшая байесовская модель - BernoulliNB
Лучшее качество - 0.9617490982664693
Параметры - {'vectorizer__stop_words': None, 'vectorizer__ngram_range': (1, 2), 'vectorizer__min_df': 10, 'vectorizer__max_df': 0.9, 'classifier__fit_prior': False, 'classifier__alpha': 1.0}


## Лучшая модель

In [32]:
train

,text,rating,length,label,len
id,,,,,
16334,почти месяц наушник пока хорошо работать микро...,4,50-100,0.75,67
38246,писать огромный комментарий о тот какой они кр...,5,250-500,1.00,402
30901,при чем тут фирма вообще грязный хлипкий наушн...,1,50-100,0.00,62
71437,полный отстой правый наушник не проработать и ...,1,0-50,0.00,50
46508,прислать быстро но один наушник не работать,3,0-50,0.50,43
...,...,...,...,...,...
42881,прислать в срок упаковка на звук хороший есть ...,3,100-150,0.50,133
56629,наушник неплохой прислать хорошо упаковать вну...,4,100-150,0.75,115
41498,хороший наушник интересный дизайн и хороший звук,5,0-50,1.00,48


In [37]:
trainb =  shuffle(pd.concat([train[train['label']==0],train[train['label']==1]]),random_state=42)
trainb['label'] = trainb['label'].apply(lambda x: int(x))

In [38]:
trainb

,text,rating,length,label,len
id,,,,,
45164,наушник огонь звук отличный микрофон тоже хороший,5,0-50,1,49
32084,прислать б у всё распаковать в скотч и прислат...,1,50-100,0,70
67784,не работать,1,0-50,0,11
53797,здравствуйте прислать наушник но кейс вовсе не...,1,150-250,0,157
8325,всё очень понравиться прийти в полностью запеч...,5,150-250,1,249
...,...,...,...,...,...
69735,прислать в целость звук качественный рекомендо...,5,0-50,1,50
68683,понравиться абсолютно всё недостаток не заметить,5,0-50,1,48
56570,хороший наушник свой деньга стоять однозначно ...,5,50-100,1,100


In [40]:
X = trainb['text']
y = trainb['label']

In [41]:
score = cross_val_score(make_pipeline(CountVectorizer(), TfidfTransformer(), LinearSVC(random_state=777)), X, y, cv=5).mean()
print(f"LinearSVC - {score}")

LinearSVC - 0.9698165583612433


In [42]:
params_grid_vectorizer = {
    'vectorizer__max_df': [0.85, 0.9, 0.95, 1.0],
    'vectorizer__min_df': [1, 10, 20],
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6)],
    'vectorizer__stop_words': [stop_words, None]
}
params_grid_transformer = {
    'transformer__norm': ['l1', 'l2'],
    'transformer__smooth_idf': [True, False],
    'transformer__use_idf': [True, False],
    'transformer__sublinear_tf': [True, False]
}
params_grid_lsvc = {
    'classifier__loss': ['hinge', 'squared_hinge'],
    'classifier__max_iter': np.arange(200, 1000, 200),
    'classifier__tol': [1e-5, 1e-4, 1e-3],
    'classifier__C': [0.9, 1.0, 1.1, 1.2]
}


In [43]:
%%time
grid_search_lsvc = make_estimator(LinearSVC(random_state=777),
                                  {**params_grid_vectorizer, **params_grid_transformer, **params_grid_lsvc}, 'accuracy', X, y)
print("LinearSVC:")
print(f"Лучшее качество - {grid_search_lsvc.best_score_}")
print(f"Параметры - {grid_search_lsvc.best_params_}")

LinearSVC:
Лучшее качество - 0.9753134619231687
Параметры - {'vectorizer__stop_words': None, 'vectorizer__ngram_range': (1, 2), 'vectorizer__min_df': 1, 'vectorizer__max_df': 0.95, 'transformer__use_idf': False, 'transformer__sublinear_tf': False, 'transformer__smooth_idf': False, 'transformer__norm': 'l2', 'classifier__tol': 1e-05, 'classifier__max_iter': 200, 'classifier__loss': 'squared_hinge', 'classifier__C': 1.2}
CPU times: total: 6min 55s
Wall time: 7min 17s


In [44]:
svc = grid_search_lsvc.best_estimator_
svc.steps

[('vectorizer', CountVectorizer(max_df=0.95, ngram_range=(1, 2))),
 ('transformer', TfidfTransformer(smooth_idf=False, use_idf=False)),
 ('classifier', LinearSVC(C=1.2, max_iter=200, random_state=777, tol=1e-05))]

In [69]:
import joblib

In [70]:
svc.predict(['любить много'])

array([1], dtype=int64)

In [71]:
joblib.dump(svc,'dumpsvc.sav')

['dumpsvc.sav']

In [72]:
loaded_model = joblib.load('dumpsvc.sav')

In [74]:
loaded_model.predict(['не любить много'])

array([0], dtype=int64)